In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen


import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../input/2012_qb_weeks1_8.csv')
df1=pd.read_csv('../input/2012_qb_weeks_9_16.csv')
df3 = pd.read_csv('../input/2012_qb_week17.csv')

In [3]:
raw_qb_2012 = pd.concat([df,df1], axis=0)
raw_qb_2012 = pd.concat([raw_qb_2012, df3], axis=0)
raw_qb_2012.drop('Rk', axis =1, inplace = True)
raw_qb_2012.shape

(629, 18)

In [4]:
raw_qb_2012.head()

,Player,Pos,Week,Team,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,QB Rating,Att.1,Yds.1,Yds/Att.1,TD.1,Fantasy Points
0,Matt Ryan,QB,1,ATL,KC,23,31,74,299,10,3,0,136,3,25,8,1,32.46
1,Robert Griffin III,QB,1,WAS,NO,19,26,73,320,12,2,0,140,9,42,5,0,25.00
2,Tony Romo,QB,1,DAL,NYG,22,29,76,307,11,3,1,130,5,12,2,0,23.48
3,Aaron Rodgers,QB,1,GB,SF,30,44,68,303,7,2,1,93,5,27,5,0,22.82
4,Drew Brees,QB,1,NO,WAS,24,52,46,339,7,3,2,71,0,0,0,0,21.56


In [16]:
# url=('https://www.foxsports.com/nfl/story/2012-nfl-full-regular-season-schedule-041712')
# html = urlopen(url)
# bsobj= BeautifulSoup(html, 'lxml')

In [17]:
# dates = soup.find_all("div", class_="flex-article-content")


In [5]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [6]:
team_stats = raw_qb_2012.groupby(['Player','Week','Opp']).sum()

In [7]:
team_stats.head(), team_stats.shape

(                        Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week Opp                                                     
 Aaron Rodgers 1    SF     30   44   68  303        7   2    1         93   
               2    CHI    22   32   69  219        7   1    1         85   
               3    SEA    26   39   67  223        6   0    0         81   
               4    NO     31   41   76  319        8   4    1        120   
               5    IND    20   32   63  235        7   3    1        103   
 
                         rush_att  rush_yds  rush_yd/att  rush_td  \
 Player        Week Opp                                             
 Aaron Rodgers 1    SF          5        27            5        0   
               2    CHI         3        -6           -2        0   
               3    SEA         2        17            9        0   
               4    NO          5        13            3        0   
               5    IND         5        57  

In [8]:
# Define a dataframe to contain the current year averages of all 2012 QBs
def create_current_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('avg_{}'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [9]:
qb_current_2012 = create_current_dataframe(team_stats)

In [23]:
# Populate current dataframe with current year averages of players
# DO NOT CHANGE!!!!!!
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    points=[]  # Create an empty list to hold the season average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total =[] # create an empty list to hold the weekly attributes value
        for week in team_stats.loc[player].index: # Iterate throough every week for the current player
            total.append(team_stats.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
            points.append(np.mean(total))  # Add the average of the total list to the season average list
    qb_current_2012['avg_{}'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    

In [24]:
qb_current_2012.head()

avg_comp    avg_att  avg_pct     avg_yds  avg_yds/att  \
Player        Week Opp                                                          
Aaron Rodgers 1    SF      30.00  44.000000     68.0  303.000000     7.000000   
              2    CHI     26.00  38.000000     68.5  261.000000     7.000000   
              3    SEA     26.00  38.333333     68.0  248.333333     6.666667   
              4    NO      27.25  39.000000     70.0  266.000000     7.000000   
              5    IND     25.80  37.600000     68.6  259.800000     7.000000   

                        avg_td   avg_int  avg_qb rating  avg_rush_att  \
Player        Week Opp                                                  
Aaron Rodgers 1    SF     2.00  1.000000      93.000000      5.000000   
              2    CHI    1.50  1.000000      89.000000      4.000000   
              3    SEA    1.00  0.666667      86.333333      3.333333   
              4    NO     1.75  0.750000      94.750000      3.750000   
              5    IND    2.00  0.800000      96.400000      4.000000   

                        avg_rush_yds  avg_rush_yd/att  avg_rush_td  \
Player        Week Opp                                               
Aaron Rodgers 1    SF      27.000000             5.00          0.0   
              2    CHI     10.500000             1.50          0.0   
              3    SEA     12.666667             4.00          0.0   
              4    NO      12.750000             3.75          0.0   
              5    IND     21.600000             5.20          0.0   

                        avg_fantasy points  
Player        Week Opp                      
Aaron Rodgers 1    SF            22.820000  
              2    CHI           16.490000  
              3    SEA           14.533333  
              4    NO            17.915000  
              5    IND           19.352000